# Sample Query Runner

Dirancang oleh: Tim Skulite  
  
Program ini akan berisi contoh penerapan pengambilan data secara lokal menggunakan Apache Jena Fuseki dan remote menggunakan data dari DBPedia / Wikidata

## Query dari Server Lokal (Jena Fuseki)

Jena Fuseki mendukung protokol HTTP untuk mengambil data yang ada didalamnya. Anggaplah Jena Fuseki seperti SQL Server (PostgreSQL, MySQL, lainnya) dan berkas berformat .ttl adalah berkas databasenya (misalnya .sqlite).  
  
Pertama harus dijalankan terlebih dahulu Jena Fusekinya. Setelah itu bisa mengambil datanya via HTTP dan kembaliannya adalah sebuah response dengan body-nya adalah datanya. Untuk menjalankan Jena Fuseki bisa menggunakan perintah di terminal:

```
cd fuseki/
./fuseki-server --file=../dataset/movies.ttl /ds
```

Proses ini akan membentuk in-memory instance dan tentunya hanya mendukung operasi READ (belum mencoba operasi lainnya). Setelah menjalankan Jena Fuseki, silakan lanjutkan menjalankan cell dibawah.

In [1]:
import json
import requests

In [2]:
# Fungsi untuk menjalankan query
local_server_url = 'http://localhost:3030/ds/query'

def get_response_from_local(query_to_send):
    try:
        response = requests.post(local_server_url, data={'query': query_to_send})
        return response.json()['results']['bindings']
    except ValueError:
        print("JSON Bermasalah atau Jena Fuseki Return 400")
        return None
    except:
        print("Jena Fuseki belum dijalankan")
        return None

In [3]:
# Contoh query mendapatkan triple S P O dari server lokal
get_response_from_local('SELECT * {?s ?p ?o} LIMIT 5')

[{'s': {'type': 'uri', 'value': 'http://skulite.org/snr/The_L_Word'},
  'p': {'type': 'uri', 'value': 'http://skulite.org/snp/country'},
  'o': {'type': 'literal', 'value': 'Canada, United States'}},
 {'s': {'type': 'uri', 'value': 'http://skulite.org/snr/The_L_Word'},
  'p': {'type': 'uri', 'value': 'http://skulite.org/snp/releaseYear'},
  'o': {'type': 'literal', 'value': '2009'}},
 {'s': {'type': 'uri', 'value': 'http://skulite.org/snr/The_L_Word'},
  'p': {'type': 'uri', 'value': 'http://skulite.org/snp/category'},
  'o': {'type': 'literal', 'value': 'TV-Show'}},
 {'s': {'type': 'uri', 'value': 'http://skulite.org/snr/The_L_Word'},
  'p': {'type': 'uri', 'value': 'http://skulite.org/snp/title'},
  'o': {'type': 'literal', 'value': 'The L Word'}},
 {'s': {'type': 'uri', 'value': 'http://skulite.org/snr/The_L_Word'},
  'p': {'type': 'uri', 'value': 'http://skulite.org/snp/dateAdded'},
  'o': {'type': 'literal', 'value': '2018-03-20'}}]

## Query dari Server Remote (DBpedia / Wikidata)

Pengambilan data secara remote dapat menggunakan bantuan library bernama RDFLib. Pada contoh kali ini, akan dilakukan pengambilan data Michael Jackson dari Wikidata

In [4]:
!pip install rdflib

In [5]:
# Import library yang diperlukan
from rdflib import Graph, URIRef
from rdflib.namespace import RDFS

In [6]:
# Melakukan import terhadap suatu berkas entity yang akan menjadi graf
g = Graph()
g.parse('https://www.wikidata.org/wiki/Special:EntityData/Q2831.ttl')

<Graph identifier=Na631bb72d3f14677859e0562b59068dd (<class 'rdflib.graph.Graph'>)>

In [7]:
# Banyaknya elemen
len(g)

110940

In [8]:
# Menjalankan query sederhana
qres = g.query(\
'''
SELECT ?label
WHERE {
wd:Q2831 skos:altLabel ?label .
}
LIMIT 5
''')

for label, *_ in qres:
    print(label.value, "<->", label.language)

Michael Joe Jackson <-> et
Regele muzicii pop <-> ro
邁克爾傑克遜 <-> zh-hant
Michael Joe Jackson <-> smj
MJ <-> de


## Query Secara Gabungan

Pada bagian ini akan dicoba mengambil data film yang sudah ada di dataset lokal, kemudian menambahkan informasi yang belum ada dari sumber remote yang kemudian hasil akhir pengolahan akan ditampilkan ke pengguna  
  
Ada 4 tahap yang akan coba dicover

1. Mengambil beberapa film + id nya yang tersedia secara lokal
2. Menampilkan sedikit detail dari film dataset lokal berdasarkan ID
3. Menampilkan film tertentu secara remote berdasarkan ID dari data lokal
4. Mengambil informasi yang  kurang dari film tersebut secara remote
5. Menampilkan kepada pengguna

In [9]:
# Mendefinisikan URI untuk query (INI PENTING)
PREFIXES = \
'''
PREFIX snr: <http://skulite.org/snr/>
PREFIX snp: <http://skulite.org/snp/>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
'''

In [10]:
# 1. Mengambil beberapa film + id nya dari dataset lokal

query = \
'''
SELECT *
WHERE { 
    ?individual rdf:type snr:Content .
    ?individual snp:id ?id
}
LIMIT 5
'''

get_response_from_local(PREFIXES + query)

[{'individual': {'type': 'uri', 'value': 'http://skulite.org/snr/The_L_Word'},
  'id': {'type': 'literal', 'value': '70136114'}},
 {'individual': {'type': 'uri',
   'value': 'http://skulite.org/snr/A_Murder_in_the_Park'},
  'id': {'type': 'literal', 'value': '80044562'}},
 {'individual': {'type': 'uri', 'value': 'http://skulite.org/snr/Collateral'},
  'id': {'type': 'literal', 'value': '80185171'}},
 {'individual': {'type': 'uri', 'value': 'http://skulite.org/snr/Jailbreak'},
  'id': {'type': 'literal', 'value': '80990658'}},
 {'individual': {'type': 'uri', 'value': 'http://skulite.org/snr/Good_Witch'},
  'id': {'type': 'literal', 'value': '80036416'}}]

In [11]:
# 2. Menampilkan sedikit detail dari film dataset lokal berdasarkan ID. Contoh: use case ketika user klik salah satu film dengan ID: 70136114

query = \
'''
SELECT ?title ?description ?releaseYear ?category
WHERE { 
    ?individual snp:id "70136114" ;
    snp:title ?title ;
    snp:description ?description ;
    snp:releaseYear ?releaseYear ;
    snp:category ?category .
}
'''

get_response_from_local(PREFIXES + query)

[{'title': {'type': 'literal', 'value': 'The L Word'},
  'description': {'type': 'literal',
   'value': 'This trailblazing series follows the tangled lives and ever-changing relationships among a close-knit group of lesbians who call Los Angeles home.'},
  'releaseYear': {'type': 'literal', 'value': '2009'},
  'category': {'type': 'literal', 'value': 'TV-Show'}}]

In [12]:
# 3. Menampilkan film tertentu secara remote berdasarkan ID dari data lokal
# SEGERA HADIR

In [13]:
# 4. Mengambil informasi yang  kurang dari film tersebut secara remote
# SEGERA HADIR

In [14]:
# 5. Menampilkan kepada pengguna
# SEGERA HADIR